<a href="https://colab.research.google.com/github/sunandhini96/TSAI_ERAV1/blob/main/LLM_from_scratch/llm_from_scratc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/llm_from_scratch

/content/drive/MyDrive/llm_from_scratch


In [ ]:
!unzip redpajama_100MB.zip -d redpajama_100mb

In [6]:
!git clone https://github.com/sunandhini96/TSAI_ERAV1.git

Cloning into 'TSAI_ERAV1'...
remote: Enumerating objects: 1331, done.
remote: Counting objects: 100% (602/602), done.
remote: Compressing objects: 100% (278/278), done.
remote: Total 1331 (delta 376), reused 516 (delta 316), pack-reused 729
Receiving objects: 100% (1331/1331), 31.01 MiB | 14.08 MiB/s, done.
Resolving deltas: 100% (621/621), done.
Updating files: 100% (186/186), done.


In [3]:
cd /content/drive/MyDrive/llm_from_scratch/TSAI_ERAV1/LLM_from_scratch

/content/drive/MyDrive/llm_from_scratch/TSAI_ERAV1/LLM_from_scratch


In [4]:
!pip install -r requirements.txt

  Cloning https://github.com/Lightning-AI/lightning (to revision 6dfa5cca9de5c28548eef5582a53c483b0eda66a) to /tmp/pip-install-c8wif7ve/lightning_b0fa33dbaa044711a5231b44a44ea674
  Running command git clone --filter=blob:none --quiet https://github.com/Lightning-AI/lightning /tmp/pip-install-c8wif7ve/lightning_b0fa33dbaa044711a5231b44a44ea674
  Running command git rev-parse -q --verify 'sha^6dfa5cca9de5c28548eef5582a53c483b0eda66a'
  Running command git fetch -q https://github.com/Lightning-AI/lightning 6dfa5cca9de5c28548eef5582a53c483b0eda66a
  Running command git checkout -q 6dfa5cca9de5c28548eef5582a53c483b0eda66a
  Resolved https://github.com/Lightning-AI/lightning to commit 6dfa5cca9de5c28548eef5582a53c483b0eda66a
  Running command git submodule update --init --recursive -q
  Encountered 22 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .no

In [6]:
!pip install gradio-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.4 MB/s eta 0:00:00


In [7]:
import torch
import torchaudio
import glob
import math
import sys
import time
from pathlib import Path
from typing import Optional, Tuple, Union
import os
print(os.getcwd())
print(torch.cuda.is_available())
print(torch.__version__)
print(torch.version.cuda)
print(torchaudio.__version__)
if torch.cuda.is_available():
    print("Number of CUDA devices:", torch.cuda.device_count())

import lightning as L
from lightning.fabric.loggers import CSVLogger
from lightning.fabric.strategies import FSDPStrategy
from torch.utils.data import DataLoader

from torch.utils.flop_counter import FlopCounterMode
from speed_monitor import SpeedMonitorBase, estimate_flops, measure_flops
from speed_monitor import SpeedMonitorFabric as SpeedMonitor
from model import GPT, Block, Config
from packed_dataset import CombinedDataset, PackedDataset, PhiAPIDataset
from utils import (
    chunked_cross_entropy,
    get_default_supported_precision,
    num_parameters,load_checkpoint
)

/content/drive/MyDrive/llm_from_scratch/TSAI_ERAV1/LLM_from_scratch
True
2.1.0+cu121
12.1
2.1.0+cu121
Number of CUDA devices: 1


In [10]:
# Configuration for running the model
model_name = "phi-1_5"
name = "redpajama"
out_dir = Path("out") / name
save_interval = 1000
eval_interval = 1000
eval_iters = 100
log_interval = 100

# Hyperparameters
learning_rate = 6e-3
batch_size = 2
micro_batch_size = 1
gradient_accumulation_steps = 4 #batch_size // micro_batch_size
assert gradient_accumulation_steps > 0
#max_iters = 600000  # num_epochs * (epoch_size // micro_batch_size) // devices
max_iters = 10000
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0
decay_lr = True
warmup_iters = 2000
lr_decay_iters = max_iters
min_lr = 6e-6

# Data configurations
data_config = [
    # ("arxiv", 2.5),
    ("book", 40.0),
    ("c4", 20.0),
    ("cc", 40.0),
    # ("github", 4.5),
    # ("stackexchange", 2.0),
    # ("wikipedia", 4.5),
]

In [11]:
hparams = {k: v for k, v in locals().items() if isinstance(v, (int, float, str)) and not k.startswith("_")}
logger = CSVLogger("out", name, flush_logs_every_n_steps=log_interval)

def setup(
    devices: int = 4,
    train_data_dir: Path = Path("data/redpajama_sample"),
    val_data_dir: Optional[Path] = None,
    precision: Optional[str] = None,
    resume: Union[bool, Path] = False,
) -> None:
    precision = precision or get_default_supported_precision(training=True)

    if devices > 1:
        strategy = FSDPStrategy(
            auto_wrap_policy={Block},
            activation_checkpointing_policy={Block},
            state_dict_type="full",
            limit_all_gathers=True,
            cpu_offload=False,
        )
    else:
        strategy = "auto"

    print(f'strategy is {strategy}')
    fabric = L.Fabric(devices=devices, strategy=strategy, precision=precision, loggers=logger)
    fabric.print(hparams)
    fabric.launch(main, train_data_dir, val_data_dir, resume)

model_copy = None

In [12]:
def main(fabric: L.Fabric, train_data_dir: Path, val_data_dir: Path, resume: Union[bool, Path]) -> None:
    global model_copy
    speed_monitor = SpeedMonitor(fabric, window_size=50, time_unit="seconds")

    if fabric.global_rank == 0:
        out_dir.mkdir(parents=True, exist_ok=True)

    config = Config.from_name(model_name)

    train_dataloader, val_dataloader = create_dataloaders(
        batch_size=micro_batch_size,
        block_size=config.block_size,
        fabric=fabric,
        train_data_dir=train_data_dir,
        val_data_dir=val_data_dir,
        seed=(1337 + fabric.global_rank),
    )
    if val_dataloader is None:
        train_dataloader = fabric.setup_dataloaders(train_dataloader)
    else:
        train_dataloader, val_dataloader = fabric.setup_dataloaders(train_dataloader, val_dataloader)

    fabric.seed_everything(1337)  # same seed for every process to init model (FSDP)

    fabric.print(f"Loading model with {config.__dict__}")
    t0 = time.perf_counter()
    import torch
    import torch.nn as nn
    def _init_weights(module: nn.Module) -> None:
            """Meant to be used with `gpt.apply(gpt._init_weights)`."""
            if isinstance(module, nn.Linear):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
                if module.bias is not None:
                    torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    with fabric.init_module(empty_init=True):
        model = GPT(config)
        model.apply(_init_weights)
    model.apply(_init_weights)


    # checkpoint_path = Path("out/redpajama/iter-000999-ckpt.pth")

    # load_checkpoint(fabric, model, checkpoint_path)

    # print(model.transformer.h[0].mlp.fc.weight)

    fabric.print(f"Time to instantiate model: {time.perf_counter() - t0:.02f} seconds.")
    fabric.print(f"Total parameters {num_parameters(model):,}")

    model = fabric.setup(model)
    optimizer = torch.optim.AdamW(
        model.parameters(), lr=learning_rate, weight_decay=weight_decay, betas=(beta1, beta2), foreach=False
    )

    # model_copy = model

    optimizer = fabric.setup_optimizers(optimizer)

    state = {"model": model, "optimizer": optimizer, "hparams": hparams, "iter_num": 0, "step_count": 0}

    if resume is True:
        resume = max(out_dir.glob("*.pth"), key=lambda p: int(p.name.split("-")[1]))
        fabric.print(f"Resume is true")
    if resume:
        fabric.print(f"Resuming training from {resume}")
        fabric.load(resume, state)

    train_time = time.perf_counter()
    train(fabric, state, train_dataloader, val_dataloader, speed_monitor)
    # train_check(fabric, state, train_dataloader, val_dataloader, speed_monitor)
    fabric.print(f"Training time: {(time.perf_counter()-train_time):.2f}s")
    if fabric.device.type == "cuda":
        fabric.print(f"Memory used: {torch.cuda.max_memory_allocated() / 1e9:.02f} GB")


In [13]:
def train_check(
    fabric: L.Fabric,
    state: dict,
    train_dataloader: DataLoader,
    val_dataloader: DataLoader,
    speed_monitor: SpeedMonitorBase,
) -> None:
    model = state["model"]
    optimizer = state["optimizer"]

    if val_dataloader is not None:
        validate(fabric, model, val_dataloader)  # sanity check

    with torch.device("meta"):
        meta_model = GPT(model.config)
        # "estimated" is not as precise as "measured". Estimated is optimistic but widely used in the wild.
        # When comparing MFU or FLOP numbers with other projects that use estimated FLOPs,
        # consider passing `SpeedMonitor(flops_per_batch=estimated_flops)` instead
        estimated_flops = estimate_flops(meta_model) * micro_batch_size
        fabric.print(f"Estimated TFLOPs: {estimated_flops * fabric.world_size / 1e12:.2f}")
        x = torch.randint(0, 1, (micro_batch_size, model.max_seq_length))
        measured_flops = measure_flops(meta_model, x)
        fabric.print(f"Measured TFLOPs: {measured_flops * fabric.world_size / 1e12:.2f}")
        del meta_model, x

    total_lengths = 0
    total_t0 = time.perf_counter()
    # print(f'model : {model}')
    for state["iter_num"], train_data in enumerate(train_dataloader, state["iter_num"]):
        print(f'train_data : {train_data.shape}')
        if state["iter_num"] >= 1:
            print(f'Breaking...')
            break

In [14]:
def train(
    fabric: L.Fabric,
    state: dict,
    train_dataloader: DataLoader,
    val_dataloader: DataLoader,
    speed_monitor: SpeedMonitorBase,
) -> None:
    model = state["model"]
    optimizer = state["optimizer"]

    if val_dataloader is not None:
        validate(fabric, model, val_dataloader)  # sanity check

    with torch.device("meta"):
        meta_model = GPT(model.config)
        # "estimated" is not as precise as "measured". Estimated is optimistic but widely used in the wild.
        # When comparing MFU or FLOP numbers with other projects that use estimated FLOPs,
        # consider passing `SpeedMonitor(flops_per_batch=estimated_flops)` instead
        estimated_flops = estimate_flops(meta_model) * micro_batch_size
        fabric.print(f"Estimated TFLOPs: {estimated_flops * fabric.world_size / 1e12:.2f}")
        x = torch.randint(0, 1, (micro_batch_size, model.max_seq_length))
        measured_flops = measure_flops(meta_model, x)
        fabric.print(f"Measured TFLOPs: {measured_flops * fabric.world_size / 1e12:.2f}")
        del meta_model, x

    total_lengths = 0
    total_t0 = time.perf_counter()

    for state["iter_num"], train_data in enumerate(train_dataloader, state["iter_num"]):
        if state["iter_num"] >= max_iters:
            checkpoint_path = f"iter-{state['iter_num']:06d}-ckpt.pth"
            fabric.print(f"Breaking at max...iter {state['iter_num']} step {state['step_count']}: loss {loss.item():.4f}")
            fabric.print(f"Saving checkpoint to {str(checkpoint_path)!r}")
            fabric.save(checkpoint_path, state)
            break

        # determine and set the learning rate for this iteration
        lr = get_lr(state["iter_num"]) if decay_lr else learning_rate
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr

        iter_t0 = time.perf_counter()

        input_ids = train_data[:, 0 : model.max_seq_length].contiguous()
        targets = train_data[:, 1 : model.max_seq_length + 1].contiguous()

        is_accumulating = (state["iter_num"] + 1) % gradient_accumulation_steps != 0
        with fabric.no_backward_sync(model, enabled=is_accumulating):
            logits = model(input_ids)
            loss = chunked_cross_entropy(logits, targets, chunk_size=0)
            fabric.backward(loss / gradient_accumulation_steps)

        # return

        if not is_accumulating:
            # print(f'state["iter_num"] : {state["iter_num"]}, gradient_accumulation_steps : {gradient_accumulation_steps}')
            # fabric.clip_gradients(model, optimizer, max_norm=grad_clip)#, norm_type="inf")
            optimizer.step()
            optimizer.zero_grad()
            state["step_count"] += 1

        t1 = time.perf_counter()
        total_lengths += input_ids.size(1)
        speed_monitor.on_train_batch_end(
            (state["iter_num"] + 1) * micro_batch_size,
            t1 - total_t0,
            # this assumes that device FLOPs are the same and that all devices have the same batch size
            fabric.world_size,
            flops_per_batch=measured_flops,
            lengths=total_lengths,
        )
        if state["iter_num"] % log_interval == 0:
            fabric.print(
                f"iter {state['iter_num']} step {state['step_count']}: loss {loss.item():.4f}, LR: {lr:.6f}, iter time:"
                f" {(t1 - iter_t0) * 1000:.2f}ms{' (optimizer.step)' if not is_accumulating else ''}"
            )

        if val_dataloader is not None and not is_accumulating and state["step_count"] % eval_interval == 0:
            t0 = time.perf_counter()
            val_loss = validate(fabric, model, val_dataloader)
            t1 = time.perf_counter() - t0
            speed_monitor.eval_end(t1)
            fabric.print(f"step {state['iter_num']}: val loss {val_loss.item():.4f}, val time: {t1 * 1000:.2f}ms")
            fabric.barrier()
        if not is_accumulating and state["step_count"] % save_interval == 0:
            checkpoint_path = f"iter-{state['iter_num']:06d}-ckpt.pth"
            fabric.print(f"{state['iter_num']} - Saving checkpoint to {str(checkpoint_path)!r}")
            fabric.save(checkpoint_path, state)
            fabric.print(
                f"iter {state['iter_num']} step {state['step_count']}: loss {loss.item():.4f}, LR: {lr:.6f}, iter time:"
            )

@torch.inference_mode()
def validate(fabric: L.Fabric, model: torch.nn.Module, val_dataloader: DataLoader) -> torch.Tensor:
    fabric.print("Validating ...")
    model.eval()

    losses = torch.zeros(eval_iters, device=fabric.device)
    for k, val_data in enumerate(val_dataloader):
        input_ids = val_data[:, 0 : model.max_seq_length].contiguous()
        targets = val_data[:, 1 : model.max_seq_length + 1].contiguous()
        logits = model(input_ids)
        losses[k] = chunked_cross_entropy(logits, targets, chunk_size=0)
    out = losses.mean()

    model.train()
    return out

In [15]:
def create_dataloader(
    batch_size: int, block_size: int, data_dir: Path, fabric: L.Fabric, shuffle: bool = True, seed: int = 12345
) -> DataLoader:
    datasets = []
    for prefix, _ in data_config:
        filenames = glob.glob(str(data_dir / f"{prefix}*"))
        dataset = PackedDataset(
            filenames,
            n_chunks=1,
            block_size=block_size,
            shuffle=shuffle,
            seed=seed,
            num_processes=fabric.world_size,
            process_rank=fabric.global_rank,
        )
        datasets.append(dataset)

    if not datasets:
        raise RuntimeError(
            f"No data found at {data_dir}. Make sure you ran prepare_redpajama.py to create the dataset."
        )

    weights = [weight for _, weight in data_config]
    sum_weights = sum(weights)
    weights = [el / sum_weights for el in weights]

    combined_dataset = CombinedDataset(datasets=datasets, seed=seed, weights=weights)

    return DataLoader(combined_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

def create_dataloaders(
    batch_size: int,
    block_size: int,
    fabric: L.Fabric,
    train_data_dir: Path = Path("data/redpajama_sample"),
    val_data_dir: Optional[Path] = None,
    seed: int = 12345,
) -> Tuple[DataLoader, DataLoader]:
    # Increase by one because we need the next word as well
    effective_block_size = block_size + 1
    train_dataloader = create_dataloader(
        batch_size=batch_size,
        block_size=effective_block_size,
        fabric=fabric,
        data_dir=train_data_dir,
        shuffle=True,
        seed=seed,
    )
    val_dataloader = (
        create_dataloader(
            batch_size=batch_size,
            block_size=effective_block_size,
            fabric=fabric,
            data_dir=val_data_dir,
            shuffle=False,
            seed=seed,
        )
        if val_data_dir
        else None
    )
    return train_dataloader, val_dataloader


In [16]:
def get_lr(it: int) -> float:
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))  # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

torch.set_float32_matmul_precision("medium")
print(f'About to run...')
print(os.getcwd())
# setup(
#     devices=1,
#     train_data_dir=Path("C://Users//rajes//Downloads//data//data//lit-redpajama-sample")
# )

About to run...
/content/drive/MyDrive/llm_from_scratch/TSAI_ERAV1/LLM_from_scratch


In [17]:
#torch.set_float32_matmul_precision("medium")
torch.cuda.amp.autocast(enabled=True)

setup(
    devices=1,
    train_data_dir=Path("/content/drive/MyDrive/llm_from_scratch/redpajama_100mb/redpajama_100MB")
)

INFO: Using bfloat16 Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO: Seed set to 1337
INFO:lightning.fabric.utilities.seed:Seed set to 1337


strategy is auto
{'model_name': 'phi-1_5', 'name': 'redpajama', 'save_interval': 1000, 'eval_interval': 1000, 'eval_iters': 100, 'log_interval': 100, 'learning_rate': 0.006, 'batch_size': 2, 'micro_batch_size': 1, 'gradient_accumulation_steps': 4, 'max_iters': 10000, 'weight_decay': 0.1, 'beta1': 0.9, 'beta2': 0.95, 'grad_clip': 1.0, 'decay_lr': True, 'warmup_iters': 2000, 'lr_decay_iters': 10000, 'min_lr': 6e-06}
Loading model with {'name': 'phi-1_5', 'hf_config': {'org': 'microsoft', 'name': 'phi-1_5'}, 'block_size': 2048, 'vocab_size': 50257, 'padding_multiple': 512, 'padded_vocab_size': 51200, 'n_layer': 24, 'n_head': 32, 'n_embd': 2048, 'rotary_percentage': 0.5, 'parallel_residual': True, 'bias': True, 'lm_head_bias': True, 'n_query_groups': 32, 'shared_attention_norm': True, '_norm_class': 'LayerNorm', 'norm_eps': 1e-05, '_mlp_class': 'GptNeoxMLP', 'gelu_approximate': 'tanh', 'intermediate_size': 8192, 'rope_condense_ratio': 1, 'rope_base': 10000, 'n_expert': 0, 'n_expert_per_tok